In [1]:
from Crypto.PublicKey import RSA
import socket
import MCipher

In [ ]:

# Key generate
prikey = RSA.generate(1024)
pubkey = prikey.publickey()

priFile = open("./serverSignPriKey.pem" , "wb+")
priFile.write(prikey.exportKey("PEM"))
priFile.close()

pubFile = open("./serverSignPubKey.pem" , "wb+")
pubFile.write(pubkey.exportKey("PEM"))
pubFile.close()

In [ ]:

def server_program() :
    host = '127.0.0.1'
    port = 5461
    
    key = 'thisisbadkeyokeythisisbadkeyokey'
    iv = 'ivisintialvector'
    
    server_socket = socket.socket()
    server_socket.bind((host, port))
    server_socket.listen(2)
    conn, address = server_socket.accept()
    
    client_pubkey = MCipher.readPEM("clientSignPubKey.pem")
    key_encrypt = MCipher.RSAEncrypt(client_pubkey , key)[0]

    conn.send(key_encrypt)
    print(conn.recv(1024).decode())
    
    conn.send(iv.encode())
    print(conn.recv(1024).decode())
    print("Connection from :" + str(address))
    
    AES = MCipher.setAES(key , iv)
    
    server_prikey = MCipher.readPEM("serverSignPriKey.pem")
    while True :
        rdata = conn.recv(1024)
        if not rdata :
            break
        d_data = MCipher.AES_Decrypt(AES,rdata)
        data, sign_data = MCipher.separateHashBlcok(d_data)
        
        if MCipher.verify(client_pubkey, data , sign_data ) : 
            print("Client가 보낸 메시지가 맞습니다. : " , data)
            message = input('->')
            block = MCipher.makeHashBlock(message, server_prikey) 
            block_en = MCipher.AES_Encrypt(AES,block)
            conn.send(block_en)
        else : 
            print("Client가 보내지 않았습니다.")
            break
            
            

    conn.close()
    
if __name__ == '__main__' :
    server_program()
    

key exchange Success
iv exchange Success
Connection from :('127.0.0.1', 65279)
Client가 보낸 메시지가 맞습니다. :  aaa
->bbbb
Client가 보낸 메시지가 맞습니다. :  cccc
